In [2]:
# Now I need to recalculate lake metadata (weights ect) in the same manner as the catchments

import ECCO_functions_v2 as ECCO


In [5]:
import pandas as pd
import osgeo.ogr
import sys, time, os, json, glob
import numpy as np
import pyproj
import h5py
import csv  # <<-- dont forget to add this ti function file
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
def MT_Gen_SWeights(nc_path):
    '''
    Purpose:          
    This program Generates metadata files used to speed up the final runs.
    This was forthe lakes (not catchments), and created a metadata text
    file, to be used as a mini-database in pandas.
    '''

        
    clim_dat,rlat,rlon,time,metadata,txtfname = ECCO.Read_CORDEX_V2(nc_path)
    vname, m1, m2, dexp, m3, m4, m5, m6, drange_orignial = metadata 
 
    lake_file = 'Lakes/ecco-biwa_lakes_v.0.2.shp'

        
    #num = lake_num
    #orog = ECCO.Height_CORDEX()
    ShapeData = osgeo.ogr.Open(lake_file)
    TheLayer = ShapeData.GetLayer(iLayer=0)
    dolakes=range(TheLayer.GetFeatureCount())
    
    for num in dolakes[0:10]:
        feature1 = TheLayer.GetFeature(num) 
        lake_feature = feature1.ExportToJson(as_object=True)
        lake_cart = ECCO.Path_LkIsl_ShpFile(lake_feature['geometry']['coordinates']) 
        EB_id = lake_feature['properties']['EBhex'][2:]
        lake_altitude=lake_feature['properties']['vfp_mean']

        lake_rprj = ECCO.Path_Reproj(lake_cart,False)

        sub_clim,sub_rlat,sub_rlon = ECCO.TrimToLake(lake_rprj,clim_dat[0,:,:],rlat,
                                                        rlon,off = 3, show = False) 
        weight_mask = ECCO.Pixel_Weights(lake_rprj,sub_clim,sub_rlat,sub_rlon)
        print EB_id,' Weight mask sum:',np.sum(weight_mask) 

        pix_truth = (weight_mask > 0.0)    # Count how many times the weight mask is
        pxnum = len(weight_mask[pix_truth])  #  above 0.0 (i.e. how many pixels of data are needed)
    
        etime = 1.0  # Nothing, can be removed (in the meta too, but be careful to change the later programs)

        ypix = -99
        xpix = -99
        if pxnum == 1:
            xxx,yyy = ECCO.Get_LatLonLim(lake_rprj.vertices)  # Find upp./low.lake lims.
            ypix = (ECCO.Closest(rlat,yyy[0]))                # For lakes of one pixel  
            xpix = (ECCO.Closest(rlon,xxx[0]))
        if pxnum < 1:
            pxnum = 1  # Small bug where it thinks lakes dont exist, no biggy...
        #print num,EB_id[2:], ECCO.Area_Lake_and_Islands(lake_cart),pxnum,etime,ypix,xpix
        
    print 'End of function'
    return 

In [18]:
ncfile= '/uio/kant/geo-metos-u1/blaken/datadisk/ECCO/CORDEX/Data_CORDEX/tas_EUR-11_ICHEC-EC-EARTH_historical_r1i1p1_KNMI-RACMO22E_v1_day_19660101-19701231.nc'

MT_Gen_SWeights(nc_path=ncfile)

a2204  Weight mask sum: 1.0
ccc59c  Weight mask sum: 1.0028562991
155980  Weight mask sum: 1.00164368292
dade22  Weight mask sum: 1.00607557045
1fc67b  Weight mask sum: 1.00410122328
663df0  Weight mask sum: 1.0
f5d07a  Weight mask sum: 1.0
232ec2  Weight mask sum: 0.983756205801
2eca07  Weight mask sum: 1.00197017198
d23e4a  Weight mask sum: 1.0
End of function
